In [20]:
# Import the required libraries and open the connection to Mongo

import collections
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats

import pymongo
from bson.objectid import ObjectId
# client = pymongo.MongoClient('mongodb://localhost:27117/')
# client = pymongo.MongoClient('mongodb://ogedei:27017')
client = pymongo.MongoClient('mongodb://localhost:27017')

In [141]:
# Connect to the database
summerofcode_db = client.summerofcode
imp = summerofcode_db.imp
icmas = summerofcode_db.icmas

In [139]:
!ls

14.1 Basic CRUD.ipynb
2016J_TMA01_q1-TN.ipynb
2016J_TMA01_q2-withTN.ipynb
import-data-Copy1.ipynb
import-data.ipynb
SUMMEROFCODE-Day 0 Warm up holiday prices-grades.csv
SUMMEROFCODE-Day 1 Choosing a holiday-grades.csv
SUMMEROFCODE-Day 2 Lifts-grades.csv
SUMMEROFCODE-Day 3 Door codes-grades.csv
SUMMEROFCODE-Day 4 Beach labyrinth-grades.csv
SUMMEROFCODE-Day 5 Laser display boards-grades.csv
TMA01_Question3_Solutions.ipynb


In [102]:
csvs = !ls *csv
csvs

['SUMMEROFCODE-Day 0 Warm up holiday prices-grades.csv',
 'SUMMEROFCODE-Day 1 Choosing a holiday-grades.csv',
 'SUMMEROFCODE-Day 2 Lifts-grades.csv',
 'SUMMEROFCODE-Day 3 Door codes-grades.csv',
 'SUMMEROFCODE-Day 4 Beach labyrinth-grades.csv',
 'SUMMEROFCODE-Day 5 Laser display boards-grades.csv']

In [114]:
def update_grades():
    for i in imp.find(modifiers={"$snapshot": True}):
        imp.update_one({'_id': i['_id']},
            {'$set': {'grade': i['Grade/10']['00'],
                      'q1': i['Q'][' 1 /5']['00'],
                      'q2': i['Q'][' 2 /5']['00']},
             '$unset': {'Grade/10': '', 'Q': ''}})

In [109]:
def update_dates():
    for i in imp.find(modifiers={"$snapshot": True}):
        starttime = datetime.strptime(i['Started on'].strip(), '%d %b %Y %H:%M')
        if i['Completed'] == '-':
            imp.update_one({'_id': i['_id']},
                           {'$set': {'started': starttime},
                            '$unset': {'Started on': '', 'Completed': '', 'Time taken': ''}})
        else:
            endtime = datetime.strptime(i['Completed'].strip(), '%d %b %Y %H:%M')
            imp.update_one({'_id': i['_id']},
                           {'$set': {'started': starttime,
                                     'completed': endtime},
                            '$unset': {'Started on': '', 'Completed': '', 'Time taken': ''}})

In [110]:
def merge_imported(icma_number):
    icmas.delete_many({'icma_number': icma_number})

    for i in imp.find(modifiers={"$snapshot": True}):
        del i['_id']
        i['icma_number'] = icma_number
        icmas.insert_one(i)

In [111]:
csvs = !ls *csv
csvs

['SUMMEROFCODE-Day 0 Warm up holiday prices-grades.csv',
 'SUMMEROFCODE-Day 1 Choosing a holiday-grades.csv',
 'SUMMEROFCODE-Day 2 Lifts-grades.csv',
 'SUMMEROFCODE-Day 3 Door codes-grades.csv',
 'SUMMEROFCODE-Day 4 Beach labyrinth-grades.csv',
 'SUMMEROFCODE-Day 5 Laser display boards-grades.csv']

In [123]:
icmas.delete_many({})

In [132]:
for csv in csvs:
    print(csv)
    qcsv = "'{}'".format(csv)
    !mongoimport --drop --db summerofcode --collection imp --type csv --headerline --ignoreBlanks --file {qcsv}
    imp.delete_many({'Surname': 'Overall average'})
    update_grades()
    update_dates()
    merge_imported(int(csv.split()[1]))

SUMMEROFCODE-Day 0 Warm up holiday prices-grades.csv
2017-07-22T15:24:32.622+0100	connected to: localhost
2017-07-22T15:24:32.622+0100	dropping: summerofcode.imp
2017-07-22T15:24:32.629+0100	imported 285 documents
SUMMEROFCODE-Day 1 Choosing a holiday-grades.csv
2017-07-22T15:24:32.971+0100	connected to: localhost
2017-07-22T15:24:32.971+0100	dropping: summerofcode.imp
2017-07-22T15:24:32.974+0100	imported 137 documents
SUMMEROFCODE-Day 2 Lifts-grades.csv
2017-07-22T15:24:33.202+0100	connected to: localhost
2017-07-22T15:24:33.202+0100	dropping: summerofcode.imp
2017-07-22T15:24:33.204+0100	imported 92 documents
SUMMEROFCODE-Day 3 Door codes-grades.csv
2017-07-22T15:24:33.394+0100	connected to: localhost
2017-07-22T15:24:33.394+0100	dropping: summerofcode.imp
2017-07-22T15:24:33.396+0100	imported 77 documents
SUMMEROFCODE-Day 4 Beach labyrinth-grades.csv
2017-07-22T15:24:33.573+0100	connected to: localhost
2017-07-22T15:24:33.573+0100	dropping: summerofcode.imp
2017-07-22T15:24:33.575+

In [134]:
icmas.find({}, ['icma']).count()

721

In [138]:
pipeline = [{"$group": {"_id": "$icma_number", "count": {"$sum": 1}}}]
list(icmas.aggregate(pipeline))

[{'_id': 3, 'count': 76},
 {'_id': 2, 'count': 91},
 {'_id': 5, 'count': 56},
 {'_id': 4, 'count': 78},
 {'_id': 1, 'count': 136},
 {'_id': 0, 'count': 284}]